In [8]:
# Install required libraries
!pip install -q faiss-cpu sentence-transformers numpy

ERROR: Could not find a version that satisfies the requirement faiss-cpu (from versions: none)
ERROR: No matching distribution found for faiss-cpu


In [12]:
!pip install -q chromadb

In [13]:
import chromadb

print(f"ChromaDB version: {chromadb.__version__}")
print("✅ ChromaDB imported successfully!")

ChromaDB version: 1.3.5
✅ ChromaDB imported successfully!


In [14]:
import chromadb
from chromadb.utils import embedding_functions
import datetime

In [19]:
import chromadb

# --- 1. Setup ---
# Create Chroma client (persistent storage)
client = chromadb.PersistentClient(path="./chroma_db")

# RESET: Delete the old collection if it exists to remove data with "string" dates
try:
    client.delete_collection(name="my_documents")
    print("✅ Deleted old collection to ensure clean state.")
except ValueError:
    pass  # Collection didn't exist, which is fine

# Create the collection
collection = client.create_collection(
    name="my_documents",
    metadata={"description": "Sample document collection"}
)

print(f"✅ Collection created: {collection.name}")
print(f"📁 Data persisted to: ./chroma_db/")

# --- 2. Add Initial Data (Base Set) ---
# NOTE: Dates are now integers (YYYYMMDD) to allow range filtering ($gte)
base_documents = [
    "Python is a versatile programming language used for web development and data science.",
    "Machine learning models require large amounts of training data to perform well.",
    "Neural networks are inspired by the structure of the human brain.",
    "Natural language processing enables computers to understand human language.",
    "Deep learning is a subset of machine learning using multi-layered neural networks."
]

base_metadatas = [
    {"category": "programming", "topic": "python", "date": 20230101, "author": "Course Team", "priority": 1},
    {"category": "AI", "topic": "machine learning", "date": 20230102, "author": "Course Team", "priority": 2},
    {"category": "AI", "topic": "neural networks", "date": 20230103, "author": "Course Team", "priority": 2},
    {"category": "AI", "topic": "NLP", "date": 20230104, "author": "Course Team", "priority": 2},
    {"category": "AI", "topic": "deep learning", "date": 20230105, "author": "Course Team", "priority": 2}
]

base_ids = [f"base_doc_{i}" for i in range(len(base_documents))]

collection.add(
    documents=base_documents,
    metadatas=base_metadatas,
    ids=base_ids
)
print(f"✅ Added {len(base_documents)} base documents.")

# --- 3. Generate & Add 30 Exercise Documents ---
# Generating diverse data with INTEGER dates
categories = ["tech", "business", "science"]
authors = ["Tech Team", "Biz Analyst", "Dr. Science"]

ex_documents = []
ex_metadatas = []
ex_ids = []

for i in range(30):
    cat = categories[i % 3]
    auth = authors[i % 3]
    
    # Generate dates: 10 in 2023, 10 in 2024, 10 in 2025
    year = 2023 + (i // 10)
    month = (i % 12) + 1
    day = (i % 28) + 1
    
    # Format date as INTEGER: YYYYMMDD (e.g., 20240115)
    date_int = int(f"{year}{month:02d}{day:02d}")
    
    # Content based on category
    if cat == "tech":
        text = f"Tech update #{i}: Python 3.{12+i} releases new asyncio features."
    elif cat == "business":
        text = f"Business report #{i}: Q{i%4+1} revenue shows {i+5}% growth in AI sector."
    else:
        text = f"Science discovery #{i}: New data on Mars mission phase {i}."

    ex_documents.append(text)
    ex_metadatas.append({
        "category": cat,
        "date": date_int, # Storing as integer
        "author": auth,
        "priority": (i % 3) + 1
    })
    ex_ids.append(f"ex_doc_{i}")

collection.add(
    documents=ex_documents,
    metadatas=ex_metadatas,
    ids=ex_ids
)
print(f"✅ Added 30 exercise documents. Total count: {collection.count()}\n")

# --- 4. Implement Filtered Queries (The Solution) ---

def run_query(query_text, where_filter=None, description=""):
    print(f"🔍 Query: '{query_text}'")
    print(f"   Filter: {description}")
    
    results = collection.query(
        query_texts=[query_text],
        n_results=3,
        where=where_filter
    )
    
    # Handle case where no results are found
    if not results['documents'][0]:
        print("   No results found.")
    else:
        for i, (doc, meta) in enumerate(zip(results['documents'][0], results['metadatas'][0]), 1):
            print(f"   {i}. {doc}")
            print(f"      Metadata: {meta}")
    print("-" * 60)

query = "latest developments"

# Task 1: Filter by Category
run_query(
    query, 
    where_filter={"category": "tech"}, 
    description="Category = 'tech'"
)

# Task 2: Filter by Author
run_query(
    query, 
    where_filter={"author": "Dr. Science"}, 
    description="Author = 'Dr. Science'"
)

# Task 3: Filter by Date Range (using $gte with INTEGER)
# We want documents from 2024-01-01 onwards -> 20240101
run_query(
    query, 
    where_filter={"date": {"$gte": 20240101}}, 
    description="Date >= 2024-01-01 (Integer: 20240101)"
)

# Task 4: Combined Filter (Category AND Date)
# Tech news from 2024 onwards
run_query(
    query, 
    where_filter={
        "$and": [
            {"category": {"$eq": "tech"}},
            {"date": {"$gte": 20240101}}
        ]
    }, 
    description="Category='tech' AND Date >= 2024"
)

✅ Deleted old collection to ensure clean state.
✅ Collection created: my_documents
📁 Data persisted to: ./chroma_db/
✅ Added 5 base documents.
✅ Added 30 exercise documents. Total count: 35

🔍 Query: 'latest developments'
   Filter: Category = 'tech'
   1. Tech update #27: Python 3.39 releases new asyncio features.
      Metadata: {'priority': 1, 'author': 'Tech Team', 'category': 'tech', 'date': 20250428}
   2. Tech update #24: Python 3.36 releases new asyncio features.
      Metadata: {'date': 20250125, 'author': 'Tech Team', 'priority': 1, 'category': 'tech'}
   3. Tech update #12: Python 3.24 releases new asyncio features.
      Metadata: {'author': 'Tech Team', 'category': 'tech', 'date': 20240113, 'priority': 1}
------------------------------------------------------------
🔍 Query: 'latest developments'
   Filter: Author = 'Dr. Science'
   1. Science discovery #8: New data on Mars mission phase 8.
      Metadata: {'category': 'science', 'priority': 3, 'author': 'Dr. Science', 'dat